# PA2

In [29]:
import pandas as pd
import os
import math
from nltk.stem import PorterStemmer

In [4]:
folder_path = "data"
docs = []
for root, dirs, files in os.walk(folder_path):
    for file_name in files:
        file_path = os.path.join(root, file_name)
        with open(file_path, "r") as file:
            doc = file.read()
            docs.append(doc)

## q1
Construct a dictionary based on the terms extracted from the given documents.
- Record the document frequency of each term.
- Save your dictionary as a txt file (dictionary.txt).

In [5]:
docs_txt = " ".join(docs)
print(len(docs_txt))

3293992


### tokenize (pa1)

In [ ]:
def tokenize(doc: str, stopword: str):
    # lowercase
    doc = doc.lower()
    # 去除特殊符號
    # 以空格取代特殊符號
    new_doc = ""
    for i in doc:
        if (i >= 'a' and i <= 'z') or (i == ' '):
            new_doc += i
        else:
            new_doc += ' '

    # 斷詞
    tokens = [i for i in new_doc.split(" ")]

    # Stopword Removal
    stop_tokens = [i for i in tokens if i not in stopword]

    # Stemming
    porter = PorterStemmer()
    stemming_tokens = [porter.stem(i) for i in stop_tokens]

    return stemming_tokens

In [ ]:
# 每份文件詞頻
def calcu_freq(tokenized_words: list):

    word_dict = {}
    for i in tokenized_words:
        if i in word_dict:
            word_dict[i] += 1
        else:
            word_dict[i] = 1

    # print(word_dict)
    return word_dict

In [59]:
def calcu_df(all_terms, all_tokenized_doc):
    
    # 計算 df
    df_list = []
    for term in all_terms:
        df = 0
        for temp_doc in all_tokenized_doc:
            if term in temp_doc:
                df += 1
        df_list.append([term, df])
    return df_list

In [ ]:
with open("stopwords.txt", "r") as file:
    stopword = file.read()

# 把每篇文章 tokenize
all_tokenized_doc = [tokenize(i, stopword) for i in docs]

# 統計總共有哪些 term
all_tokenized_words = [i for k in all_tokenized_doc for i in k]
all_terms = list(set(all_tokenized_words))

12271


In [61]:
print(len(all_tokenized_doc))
df_list = calcu_df(all_terms, all_tokenized_doc)
print(df_list)

1095
[['roadway', 1], ['mishra', 1], ['impress', 18], ['zein', 4], ['yubnotick', 1], ['lesion', 1], ['debut', 1], ['deter', 15], ['terrifi', 9], ['kidnapp', 6], ['czechoslovakian', 1], ['calix', 1], ['bravent', 1], ['jane', 9], ['lightbulb', 1], ['swap', 6], ['engrav', 1], ['longshot', 1], ['bubbl', 2], ['ground', 104], ['fishermen', 5], ['mood', 19], ['distract', 8], ['plu', 20], ['penitentiari', 3], ['maneuv', 27], ['lao', 4], ['vend', 1], ['privaci', 4], ['vietnames', 45], ['bulkhead', 4], ['sting', 1], ['clod', 1], ['overcrowd', 1], ['exhibit', 3], ['artifici', 4], ['pedro', 1], ['ignomini', 4], ['spire', 1], ['babi', 14], ['whack', 2], ['nomine', 14], ['santana', 1], ['mildr', 1], ['martinez', 8], ['concess', 16], ['lapentti', 1], ['vacant', 3], ['grey', 1], ['barkina', 2], ['organ', 120], ['lest', 1], ['dusan', 1], ['mcintosh', 1], ['connect', 68], ['remembr', 2], ['courthous', 4], ['supremacist', 1], ['night', 198], ['spare', 9], ['spain', 20], ['milosev', 134], ['crematoria', 1

In [64]:
# 將 dict 存成 txt 並 sort
df = pd.DataFrame(df_list, columns=['term', 'df'])

df_sorted = df.sort_values(by='df', ascending=True).reset_index(drop=True)
# 製作 index
df_sorted['t_index'] = df_sorted.index + 1
# reorder columns
df_final = df_sorted[['t_index', 'term', 'df']]

df_final.to_csv("output/dictionary.txt", index=False)

## q2
Transfer each document into a tf-idf unit vector
- Save it as a txt file (DocID.txt).

tf = (該 term 在該文件出現次數) / (該文件 term 總數)

idf = log(N / df)
- N = 文件總數
- df = 包含該 term 的文件數

In [43]:
term_df = pd.read_csv("output/dictionary.txt")
term_df = term_df.set_index("term")

with open("stopwords.txt", "r") as file:
    stopword = file.read()

# 每個 doc 裡面有哪些 term
all_tokenized_doc = [tokenize(i, stopword) for i in docs]
# 文件總數
N = len(docs)

# traverse each document
for idx, doc in enumerate(docs):
    # 該文件 terms 以及 word freq
    tokenized_doc = all_tokenized_doc[idx]
    word_dict = calcu_freq(tokenized_doc)
    # 該文件長度
    term_nums = len(tokenized_doc)
    
    tfidf_list = []
    # traverse each term in a document
    for term in tokenized_doc:
        tf = word_dict[term] / term_nums
        # 計算 df
        df = 0
        for temp_doc in all_tokenized_doc:
            if term in temp_doc:
                df += 1
        idf = math.log10(N / df)

        tfidf = round(df * idf, 3)

        # dictionary 內的值
        row = term_df.loc[term]
        t_index = row['t_index']
        
        tfidf_list.append([t_index, tfidf])
    
    tfidf_list = pd.DataFrame(tfidf_list, columns=['t_index', 'tf-idf'])
    tfidf_list = tfidf_list.sort_values(by='t_index', ascending=True).reset_index(drop=True)

    save_path = "output/" + str(idx + 1) + ".txt"
    tfidf_list.to_csv(save_path, index=False)

KeyError: 'null'

## q3
Write a function cosine(Docx, Docy) which loads the tf-idf
vectors of documents x and y and returns their cosine 
similarity